In [ ]:
import json
import os
from datetime import datetime, timedelta
import pandas as pd

def load_gaze_data(file_path):
    df = pd.read_csv(file_path)
    df["current_time"] = pd.to_datetime(
        df["current_time"], format="%Y-%m-%dT%H:%M:%S.%fZ"
    )
    return df


def load_json_data(file_path):
    with open(file_path, "r") as file:
        json_data = json.load(file)
    return json_data


def process_gaze_data(df, json_data):
    # Step 1: initial date and first time
    initial_date = datetime.strptime(
        json_data[0]["initialDate"], "%Y-%m-%dT%H:%M:%S.%fZ"
    )
    last_time_seconds = df[df["current_time"] - initial_date < timedelta(seconds=0)]
    if last_time_seconds.empty:
        less = df["current_time"].loc[0]
        last_time_seconds = -abs(less - initial_date).total_seconds()
    else:
        last_time_seconds = last_time_seconds["time_seconds"].iloc[-1]

    # Step 2: Remove all rows where `time_seconds` is less than 0
    df["time_seconds"] = df["time_seconds"] - last_time_seconds
    df = df[df["time_seconds"] >= 0].reset_index(drop=True)
    df["postID"] = None

    # Step 3: Assign postID based on PostStartTime and PostEndTime
    for obj in json_data:
        post_start_time = obj["PostStartTime"]
        post_end_time = obj["PostEndTime"]
        post_id = obj["postID"]

        df.loc[
            (df["time_seconds"] >= post_start_time)
            & (df["time_seconds"] <= post_end_time),
            "postID",
        ] = post_id

    # Step 4: Remove all rows where `postID` is None
    df = df[df["postID"].notna()].reset_index(drop=True)
    print("dataframe filtered", df)
    return df


def process_screenshots(screenshots_folder, json_data):

    # Step 1: Get the list of screenshot files
    screenshot_files = os.listdir(screenshots_folder)
    screenshot_files = [file for file in screenshot_files if file.endswith(".png")]
    screenshot_assignments = []

    # Step 2: Assign postID based on screenshot timestamp
    for file in screenshot_files:
        timestamp_str = (
            file.replace("screenshot_", "").replace(".png", "").replace("_", ":")
        )
        screenshot_time = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S")
        assigned_post_id = None
        for obj in json_data:
            initial_date = datetime.strptime(
                obj["initialDate"], "%Y-%m-%dT%H:%M:%S.%fZ"
            )
            post_start_time = initial_date + timedelta(seconds=obj["PostStartTime"])
            post_end_time = initial_date + timedelta(seconds=obj["PostEndTime"])

            if post_start_time <= screenshot_time <= post_end_time:
                assigned_post_id = obj["postID"]
                break

        screenshot_assignments.append(
            {
                "filename": file,
                "screenshot_time": screenshot_time,
                "postID": assigned_post_id,
            }
        )

    # Step 3: Create a DataFrame with the assignments
    screenshot_df = pd.DataFrame(screenshot_assignments)
    screenshot_df = screenshot_df[screenshot_df["postID"].notna()].reset_index(
        drop=True
    )
    screenshot_df = screenshot_df.drop_duplicates(subset="postID", keep="first")
    screenshot_df.sort_values(by="screenshot_time", inplace=True)
    return screenshot_df


def assign_screenshot_filenames(df, screenshot_df):
    screenshot_df["postID"] = screenshot_df["postID"].astype(int)
    postID_to_filename = screenshot_df.set_index("postID")["filename"].to_dict()
    df["screenshot_filename"] = df["postID"].map(postID_to_filename)
    return df


def save_split_files(df, output_folder, name):
    os.makedirs(output_folder, exist_ok=True)
    unique_post_ids = df["postID"].unique()

    for post_id in unique_post_ids:
        df_filtered = df[df['postID'] == post_id]
        filename = f'{name}_gaze_{post_id}.csv'
        df_filtered["x"] = df_filtered["x"].astype(int)
        df_filtered["y"] = df_filtered["y"].astype(int)
        df_filtered.to_csv(os.path.join(output_folder, filename), index=False)

    print(f"Archivos CSV creados en la carpeta {output_folder}")




def main():
    name = "gabriel"

    root = f"data/{name}"
    input_file = root + "gaze_clean.csv"
    json_file = root + f"times/{name}_posts_times.json"

    df = load_gaze_data(input_file)
    json_data = load_json_data(json_file)
    df = process_gaze_data(df, json_data)
    unique_post_ids = df["postID"].unique()

    collect_screenshots(unique_post_ids, name=name, root=root)




def collect_screenshots(unique_post_ids, name, root):
    for post_id in unique_post_ids:
        df_file = pd.read_csv(root + f"gaze_posts/{name}_gaze_{post_id}.csv")
        image_screenshot = (
            root + f"screenshots/{df_file['screenshot_filename'].iloc[0]}"
        )

        new_screenshot_filename = f"{name}_screenshot_{post_id}.png"
        new_screenshot_path = root + f"screenshots/{new_screenshot_filename}"

        if os.path.exists(image_screenshot):
            os.rename(image_screenshot, new_screenshot_path)